In [1]:
# pbp_parse.py
#
# parse JSON, make game structure, convert to CSV.
# JSON 데이터를 읽어와 게임 상황을 구현하고
# gameday line 하나에 맞는 text row를 생성, CSV 파일에 저장한다.

import os
import json
from collections import OrderedDict
import regex
import csv

# custom library
from utils import print_progress

%load_ext autoreload
%autoreload 2

In [163]:
import pandas as pd
game_id = '20190331SKWO02019'
pl = '_pitching.csv'
bl = '_batting.csv'
rl = '_relay.csv'

pdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{pl}')
bdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{bl}')
rdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{rl}')

In [143]:
game_date = game_id[:8]
away = game_id[8:10]
home = game_id[10:12]

In [144]:
tcol = ['textOrder', 'seqno', 'text', 'type', 'stuff', 'pitchNum',
        'pitchResult', 'pitchId', 'speed', 'referee', 'stadium', 'inn',
        'ballcount', 'stance']
pcol = ['pitchId', 'speed', 'crossPlateX',
        'vx0', 'vy0', 'vz0', 'x0', 'z0', 'ax', 'ay', 'az',
        'topSz', 'bottomSz']

`pitchResult`
- `T` : s__T__rike 스트라이크
- `F` : __F__oul 파울
- `S` : __S__wing 헛스윙
- `B` : __B__all 볼
- `H` : __H__it 타격

`type`
- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

- `pitchNum` : 문자중계 텍스트(`textOptionList`)에 포함되는 투구수, 정합성 확인 안됨, 신뢰하지 말 것
- `ballcount` : pitch data(`ptsOptionList`)에 포함되는 투구수, `ptsPitchId` 오류 있을 때는 1개씩 밀리는 문제, 신뢰하지 말 것
- `inn` : 문자중계와 pitch data에 모두 포함, 지금은 Pitch data에 있는걸 가져오는 중. 신뢰하지 말 것

In [164]:
rdf.columns

Index(['seqno', 'text', 'type', 'stuff', 'pitchNum', 'pitchResult', 'pitchId',
       'speed', 'outPlayer', 'inPlayer', 'shiftPlayer', 'textOrder', 'inn',
       'ballcount', 'crossPlateX', 'topSz', 'crossPlateY', 'vy0', 'vz0', 'vx0',
       'z0', 'y0', 'ax', 'x0', 'ay', 'az', 'bottomSz', 'stance'],
      dtype='object')

In [124]:
display(rdf.loc[(rdf.pitchNum != rdf.ballcount) & ~np.isnan(rdf.pitchNum)][tcol])
display(rdf.loc[rdf.textOrder==7][tcol])
display(rdf.loc[rdf.textOrder==8][tcol])

,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
44,7,44,3구 볼,1,직구,3.0,B,-1,145.0,우효동,고척,NaN,NaN,NaN
45,7,45,4구 볼,1,직구,4.0,B,190331_141806,146.0,우효동,고척,1.0,3.0,R
53,8,53,5구 볼,1,슬라이더,5.0,B,-1,132.0,우효동,고척,NaN,NaN,NaN
54,8,54,6구 타격,1,직구,6.0,H,190331_142138,143.0,우효동,고척,1.0,5.0,R


,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
41,7,41,3번타자 김하성,8,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
42,7,42,1구 볼,1,직구,1.0,B,190331_141656,144.0,우효동,고척,1.0,1.0,R
43,7,43,2구 볼,1,직구,2.0,B,190331_141720,146.0,우효동,고척,1.0,2.0,R
44,7,44,3구 볼,1,직구,3.0,B,-1,145.0,우효동,고척,NaN,NaN,NaN
45,7,45,4구 볼,1,직구,4.0,B,190331_141806,146.0,우효동,고척,1.0,3.0,R
46,7,46,김하성 : 볼넷,13,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
47,7,47,1루주자 이정후 : 2루까지 진루,14,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN


,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
48,8,48,4번타자 박병호,8,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
49,8,49,1구 볼,1,슬라이더,1.0,B,190331_141857,128.0,우효동,고척,1.0,1.0,R
50,8,50,2구 스트라이크,1,슬라이더,2.0,T,190331_141923,131.0,우효동,고척,1.0,2.0,R
51,8,51,3구 파울,1,직구,3.0,F,190331_141952,143.0,우효동,고척,1.0,3.0,R
52,8,52,4구 볼,1,커브,4.0,B,190331_142030,121.0,우효동,고척,1.0,4.0,R
53,8,53,5구 볼,1,슬라이더,5.0,B,-1,132.0,우효동,고척,NaN,NaN,NaN
54,8,54,6구 타격,1,직구,6.0,H,190331_142138,143.0,우효동,고척,1.0,5.0,R
55,8,55,박병호 : 우익수 플라이 아웃,13,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
56,8,56,2루주자 이정후 : 3루까지 진루,14,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN


In [125]:
display(rdf.loc[(rdf.pitchNum != rdf.ballcount) & ~np.isnan(rdf.pitchNum)][tcol])
display(rdf.loc[rdf.textOrder==7][pcol])
display(rdf.loc[rdf.textOrder==8][pcol])

,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
44,7,44,3구 볼,1,직구,3.0,B,-1,145.0,우효동,고척,NaN,NaN,NaN
45,7,45,4구 볼,1,직구,4.0,B,190331_141806,146.0,우효동,고척,1.0,3.0,R
53,8,53,5구 볼,1,슬라이더,5.0,B,-1,132.0,우효동,고척,NaN,NaN,NaN
54,8,54,6구 타격,1,직구,6.0,H,190331_142138,143.0,우효동,고척,1.0,5.0,R


,pitchId,speed,crossPlateX,vx0,vy0,vz0,x0,z0,ax,ay,az,topSz,bottomSz
41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,190331_141656,144.0,-1.381740,2.18982,-131.462,-5.13202,-1.61299,6.73315,-8.21051,32.8010,-8.81017,3.74295,1.72
43,190331_141720,146.0,-0.357717,3.91236,-132.502,-6.00184,-1.47303,6.67330,-5.29041,33.7717,-7.30470,3.74295,1.72
44,-1,145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,190331_141806,146.0,-0.330984,5.44231,-132.517,-5.83543,-1.74018,6.62273,-9.23813,31.6325,-6.40115,3.74295,1.72
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,pitchId,speed,crossPlateX,vx0,vy0,vz0,x0,z0,ax,ay,az,topSz,bottomSz
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,190331_141857,128.0,1.406350,6.18180,-116.676,-4.19521,-1.57996,6.66356,2.80635,27.6763,-22.14500,3.61588,1.7099
50,190331_141923,131.0,0.883311,5.33823,-119.518,-5.54216,-1.59618,6.57244,2.18573,27.0349,-24.39900,3.61588,1.7099
51,190331_141952,143.0,-0.609943,3.61421,-130.395,-8.72510,-1.46448,6.61490,-7.32211,33.6402,-7.36690,3.61588,1.7099
52,190331_142030,121.0,1.079340,4.53692,-110.356,-5.09509,-1.44254,6.88162,3.87766,24.2575,-34.78350,3.61588,1.7099
53,-1,132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,190331_142138,143.0,0.267466,5.98161,-129.806,-9.37733,-1.45503,6.58028,-8.10931,30.3600,-4.52085,3.61588,1.7099
55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
rdf.loc[rdf.type == 99][tcol]

,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
606,107,603,=====================================,99,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
607,107,604,승리투수: 하재훈,99,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN


In [32]:
rdf[tcol]

,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
0,0,0,1회초 SK 공격,0,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
1,1,1,1번타자 노수광,8,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
2,1,2,1구 스트라이크,1,슬라이더,1.0,T,190331_140049,137.0,우효동,고척,1.0,1.0,L
3,1,3,2구 스트라이크,1,투심,2.0,T,190331_140108,142.0,우효동,고척,1.0,2.0,L
4,1,4,3구 파울,1,체인지업,3.0,F,190331_140127,125.0,우효동,고척,1.0,3.0,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,106,600,1구 스트라이크,1,직구,1.0,T,190331_180403,145.0,우효동,고척,9.0,1.0,R
604,106,601,2구 타격,1,슬라이더,2.0,H,190331_180427,132.0,우효동,고척,9.0,2.0,R
605,106,602,이지영 : 우익수 플라이 아웃,13,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
606,107,603,=====================================,99,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
